<a href="https://colab.research.google.com/github/pollinations/hive/blob/main/interesting_notebooks/COG_Swinir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://ipfs.pollinations.ai/ipfs/QmbPCzFiLrskHNQKs5eM8CGkEQ6qvirBTmvGLkKVWRcCwX" width="300" height="300" />

Uses [SwinIR](https://github.com/JingyunLiang/SwinIR) to perform 4x neural super-resolution (creates HD images from low resolution)

In [ ]:

# The image or video to scale up
image_file = ''  #@param {type: "string"}

output_path = "/content"


In [ ]:
%cd /content
!git clone https://github.com/voodoohop/SwinIR
%cd

In [ ]:
!bash download-weights.sh


In [ ]:
!pip install "torchvision==0.9.0" "torch==1.8.0" "numpy==1.19.4" "opencv-python==4.4.0.46" "tqdm==4.62.2" "Pillow==8.3.2" "timm==0.4.12" "ipython==7.19.0"
!pip install timm

In [ ]:
#!pip install cog redis
#%cd /content/swinir

from predict import Predictor
p = Predictor()
p.setup()

path = p.predict(image_file)

In [ ]:
!cp {path} {output_path}